In [1]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    import pandas as pd
    import pandas_datareader.data as web
    import pickle 
    import requests
    import tensorflow as tf
    from collections import deque
    import random
    import os
    from datetime import date

In [2]:
name = 'Duel Q-learning agent'
image_path = 'D:/kenneth/agent/images/'
path = 'D:/kenneth/trading/'
run_date=open(path+'run_date.txt').read()
start=('2019-01-01')
end=('2019-12-31')
days=30

In [3]:
if not os.path.exists(image_path+name):
        os.makedirs(image_path+name) 
with open('D:/kenneth/trading/sp500/sp500tickers.txt','r') as f:
    tickers=[line.rstrip('\n') for line in f]

In [4]:
class Agent:
    def __init__(self, state_size, window_size, trend, skip, batch_size):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.action_size = 3
        self.batch_size = batch_size
        self.memory = deque(maxlen = 1000)
        self.inventory = []

        self.gamma = 0.95
        self.epsilon = 0.5
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999

        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.X = tf.placeholder(tf.float32, [None, self.state_size])
        self.Y = tf.placeholder(tf.float32, [None, self.action_size])
        feed = tf.layers.dense(self.X, 512, activation = tf.nn.relu)
        tensor_action, tensor_validation = tf.split(feed,2,1)
        feed_action = tf.layers.dense(tensor_action, self.action_size)
        feed_validation = tf.layers.dense(tensor_validation, 1)
        self.logits = feed_validation + tf.subtract(feed_action,tf.reduce_mean(feed_action,axis=1,keepdims=True))
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(
            self.cost
        )
        self.sess.run(tf.global_variables_initializer())

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        return np.argmax(
            self.sess.run(self.logits, feed_dict = {self.X: state})[0]
        )
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def replay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size, l):
            mini_batch.append(self.memory[i])
        replay_size = len(mini_batch)
        X = np.empty((replay_size, self.state_size))
        Y = np.empty((replay_size, self.action_size))
        states = np.array([a[0][0] for a in mini_batch])
        new_states = np.array([a[3][0] for a in mini_batch])
        Q = self.sess.run(self.logits, feed_dict = {self.X: states})
        Q_new = self.sess.run(self.logits, feed_dict = {self.X: new_states})
        for i in range(len(mini_batch)):
            state, action, reward, next_state, done = mini_batch[i]
            target = Q[i]
            target[action] = reward
            if not done:
                target[action] += self.gamma * np.amax(Q_new[i])
            X[i] = state
            Y[i] = target
        cost, _ = self.sess.run(
            [self.cost, self.optimizer], feed_dict = {self.X: X, self.Y: Y}
        )
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return cost
    
    def buy(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(state)
            next_state = self.get_state(t + 1)
            
            if action == 1 and initial_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
                #print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
                
                
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((close[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
#                 print(
#                     'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
#                     % (t, close[t], invest, initial_money)
#                 )
            
            state = next_state
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        self.sess.close()
        return states_buy, states_sell, total_gains, invest
        
    def train(self, iterations, checkpoint, initial_money):
        for i in range(iterations):
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            for t in range(0, len(self.trend) - 1, self.skip):
                action = self.act(state)
                next_state = self.get_state(t + 1)
                
                if action == 1 and starting_money >= self.trend[t] and t < (len(self.trend) - self.half_window):
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                    
                invest = ((starting_money - initial_money) / initial_money)
                self.memory.append((state, action, invest, 
                                    next_state, starting_money < initial_money))
                state = next_state
                batch_size = min(self.batch_size, len(self.memory))
                cost = self.replay(batch_size)
            if (i+1) % checkpoint == 0:
                pass
#                 print('epoch: %d, total rewards: %f.3, cost: %f, total money: %f'%(i + 1, total_profit, cost,
#                                                                                   starting_money))

In [5]:
def plot_all(ticker):
    close = df['Close']
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=10, color='g', label = 'buying signal', markevery = states_buy)
    plt.plot(close, 'v', markersize=10, color='r', label = 'selling signal', markevery = states_sell)
    plt.title(name+'\n'+'Ticker: '+ticker)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [6]:
def display_only(days):
    new_buy = []
    new_sell = []
    display_days = days
    total_days= df.shape[0]
    display_from = total_days-display_days
    for i in(states_buy):
        if i > display_from:
            adj = i- display_from
            new_buy.append(adj)
    for i in(states_sell):
        if i > display_from:
            adj = i- display_from
            new_sell.append(adj)
    
    return(new_buy, new_sell)

In [7]:
def plot_by_days(days, ticker):
    close = df['Close'][-days:]
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=12, color='g',alpha=1.0,label = 'buying signal', markevery = new_buy)
    plt.plot(close, 'v', markersize=12, color='r',alpha=1.0, label = 'selling signal', markevery = new_sell)
    plt.title(name+'\n'+'Last '+str(days)+' Daily Trade Recommendations')
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'_'+str(days)+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [8]:
def save_signals():
    path2 = image_path+name+'/signal/'
    if not os.path.exists(path2):
        os.makedirs(path2)
        
    if  (len(states_buy)!= 0) and (len(states_sell) != 0):  
        if int(states_buy[-1])>int(states_sell[-1]):
            last_signal = 'Trading Buy'
            signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
        else:
            last_signal = 'Trading Sell'
            signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
        
    elif len(states_buy)== 0 and len(states_sell)>0:
        last_signal = 'Trading Sell'
        signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)>0:
        last_signal = 'Trading Buy'
        signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)==0:
        last_signal = 'None'
        signal_day = 'None'  
        
    with open(path2+ticker+'.txt','w') as f:
        print(
            'model_run,','signal_day,','last_signal,','total_gains,','invest,','data_start','\n',
              str(date.today()),',',signal_day,',',last_signal,',',total_gains,',',invest,',',str(start),
              file=f)

In [9]:
initial_money = 10000
window_size = 30
skip = 1
iterations = 10
checkpoint = 10
batch_size = 32


for ticker in tickers[376:]:
    df = pd.read_csv(path+'sp500/'+run_date+'/'+ticker+'.csv', index_col=0, parse_dates=True)
    df =df[start:end]
    print('Started ticker:', ticker)
    
    close = df.Close.values.tolist()
   
    agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              skip = skip, 
              batch_size = batch_size)
    
    agent.train(iterations = iterations, checkpoint = checkpoint, initial_money = initial_money)

    states_buy, states_sell, total_gains, invest = agent.buy(initial_money = initial_money)
    save_signals()
    plot_all(ticker)
#     new_buy, new_sell = display_only(days)
#     plot_by_days(days, ticker)


Started ticker: PM
Started ticker: PNC
Started ticker: PNR
Started ticker: PNW
Started ticker: PPG
Started ticker: PPL
Started ticker: PRGO
Started ticker: PRU
Started ticker: PSA
Started ticker: PSX
Started ticker: PVH
Started ticker: PWR
Started ticker: PXD
Started ticker: PYPL
Started ticker: QCOM
Started ticker: QRVO
Started ticker: RCL
Started ticker: RE
Started ticker: REG
Started ticker: REGN
Started ticker: RF
Started ticker: RHI
Started ticker: RJF
Started ticker: RL
Started ticker: RMD
Started ticker: ROK
Started ticker: ROL
Started ticker: ROP
Started ticker: ROST
Started ticker: RSG
Started ticker: RTN
Started ticker: SBAC
Started ticker: SBUX
Started ticker: SCHW
Started ticker: SEE
Started ticker: SHW
Started ticker: SIVB
Started ticker: SJM
Started ticker: SLB
Started ticker: SLG
Started ticker: SNA
Started ticker: SNPS
Started ticker: SO
Started ticker: SPG
Started ticker: SPGI
Started ticker: SPY
Started ticker: SRE
Started ticker: STI
Started ticker: STT
Started ticke